In [8]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import string                              
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer 
stopwords_english = stopwords.words('english')
tokenizer = TweetTokenizer(preserve_case = False, 
                           strip_handles = True,
                           reduce_len    = True)
stemmer = PorterStemmer()
webs    = pd.ExcelFile("WEBs_act.xls")
df      = webs.parse("Categorias")
textos2 = pd.DataFrame(columns = ['Texto', 'Clase'])
for i in df.index:
    url       = df["URL"][i]
    pagina    = get(url)
    html      = BeautifulSoup(pagina.text,"html.parser")
    parrafos  = [x.text.strip() for x in html.findAll("p") if x.text != "\n"]
    for p in parrafos:
        if 50 < len(p) < 1600:
            texto_tokens = tokenizer.tokenize(p)
            texto_clean  = [ word for word in texto_tokens if
                            word not in stopwords_english and
                            word not in string.punctuation]
            texto_stem   = [stemmer.stem(word) for word in texto_clean]
            textos2 = textos2.append({'Texto': texto_stem, 'Clase':df["CÓDIGO"][i]}, ignore_index=True)

In [10]:
len(textos2)

5349

In [15]:
textos2.describe()

,Texto,Clase
count,5349,5349
unique,5348,5
top,"[war, last, mani, year, length, end, acced, te...",3
freq,2,1107


In [13]:
textos2["Clase"].value_counts()

3    1107
2    1089
5    1079
1    1054
4    1020
Name: Clase, dtype: int64

In [14]:
textos2.to_csv("historia.csv")